In [55]:
import pandas as pd

# Load the dataset
data = pd.read_csv("data/Quote-Equity-RELIANCE-EQ-31-10-2023-to-31-10-2024.csv")


# Trim column names by stripping whitespace
data.columns = data.columns.str.strip()

# Select relevant columns and set 'Date' as the index
data = data[['Date', 'OPEN', 'HIGH', 'LOW', 'close', 'VOLUME']].copy()
data.set_index('Date', inplace=True)

# Rename columns
data.rename(columns={'OPEN': 'Open', 'HIGH': 'High', 'LOW': 'Low', 'close': 'Close', 'VOLUME': 'Volume'}, inplace=True)

"""
# Remove commas and convert data types
data['Open'] = data['Open'].str.replace(',', '').astype(float)
data['High'] = data['High'].str.replace(',', '').astype(float)
data['Low'] = data['Low'].str.replace(',', '').astype(float)
data['Close'] = data['Close'].str.replace(',', '').astype(float)
"""
# Remove commas for 'Volume' column and convert to integer
data['Volume'] = data['Volume'].str.replace(',', '').astype(int)

for col in ['Open', 'High', 'Low', 'Close']:
    if data[col].dtype == 'object' and data[col].str.contains(',').any():
        data[col] = data[col].str.replace(',', '').astype(float)

data.index = pd.to_datetime(data.index).strftime('%Y-%m-%d')

# Verify changes
data.head()


,Open,High,Low,Close,Volume
Date,,,,,
2024-10-31,1340.0,1343.0,1326.15,1332.05,9331650
2024-10-30,1335.0,1350.0,1325.35,1343.90,11984423
2024-10-29,1328.1,1343.2,1320.30,1340.00,12008361
2024-10-28,1337.0,1353.0,1322.10,1334.35,10824350
2024-10-25,2687.0,2688.7,2644.00,2655.70,9298748


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251 entries, 2024-10-31 to 2023-10-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    251 non-null    float64
 1   High    251 non-null    float64
 2   Low     251 non-null    float64
 3   Close   251 non-null    float64
 4   Volume  251 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 11.8+ KB


In [57]:
data.shape

(251, 5)

In [58]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [59]:
# handling missing daa 
data = data.dropna() # drop rows with missing values

# calculate the 14-day SMA and 14-day RSI
data['SMA_14'] = data['Close'].rolling(window=14).mean()

# RSI calculation
delta = data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

# Normalize 'Close' price for model compatibility (optional)
data['Close_normalized'] = (data['Close'] - data['Close'].mean()) / data['Close'].std()

start_index = 14
data = data[start_index:]
# Display the data with new indicators
print(data[['Close', 'SMA_14', 'RSI_14', 'Close_normalized']].head(20))

              Close       SMA_14     RSI_14  Close_normalized
Date                                                         
2024-10-11  2744.20  2412.328571  95.886271         -0.201741
2024-10-10  2742.10  2512.200000  95.722695         -0.209041
2024-10-09  2749.20  2612.857143  95.986164         -0.184360
2024-10-08  2794.70  2717.167857  96.447314         -0.026189
2024-10-07  2741.45  2723.292857  64.105938         -0.211301
2024-10-04  2773.05  2729.967857  64.992780         -0.101451
2024-10-03  2813.95  2739.746429  69.557143          0.040729
2024-10-01  2929.65  2757.100000  76.636334          0.442934
2024-09-30  2953.15  2772.439286  75.096412          0.524627
2024-09-27  3052.35  2796.278571  82.897979          0.869474
2024-09-26  2995.90  2816.496429  75.365176          0.673238
2024-09-25  2987.90  2836.478571  74.922049          0.645428
2024-09-24  2978.75  2857.242857  76.412866          0.613620
2024-09-23  2986.75  2874.507143  74.107321          0.641430
2024-09-

In [60]:
# Step 1: Initialize variables for backtesting
initial_balance = 10000  # Starting with ₹10,000
balance = initial_balance
position = 0  # Start with no stock position
returns = []  # To store returns from each trade

# Step 2: Iterate through data to simulate the trading strategy
for i in range(1, len(data)):
    if data['RSI_14'].iloc[i] < 30 and position == 0:  # Buy condition
        # Buy stock
        position = balance / data['Close'].iloc[i]
        balance = 0  # Invest all balance into stock

    elif data['RSI_14'].iloc[i] > 70 and position > 0:  # Sell condition
        # Sell stock
        balance = position * data['Close'].iloc[i]
        position = 0  # No stock position after selling
        returns.append(balance - initial_balance)  # Calculate profit or loss

# Step 3: Calculate overall performance
final_balance = balance if position == 0 else position * data['Close'].iloc[-1]
total_return = final_balance - initial_balance

# Display results
print(f"Initial Balance: ₹{initial_balance}")
print(f"Final Balance: ₹{final_balance}")
print(f"Total Return: ₹{total_return}")
print(f"Return Percentage: {((final_balance - initial_balance) / initial_balance) * 100:.2f}%")


Initial Balance: ₹10000
Final Balance: ₹9020.786383100525
Total Return: ₹-979.2136168994748
Return Percentage: -9.79%
